In [0]:
# !opencv_traincascade

In [0]:
# !opencv_createsamples

In [0]:
# !opencv_traincascade

In [0]:
# монтировать диск
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# скопировать и разархивировать картинки 
!cp "/content/drive/My Drive/smpls.zip" /content
!unzip  /content/smpls.zip

In [0]:
# создать файл со списком позитивных картинок
import os
pos_dir = "pos/"

pos_list = ""
for name in os.listdir(pos_dir):
  if name.endswith(".png"):
    if len(pos_list)<1:
      pos_list =  pos_dir + name + "  1  0 0 300 30"
    else:
      pos_list = pos_list + "\r\n" +  pos_dir + name + "  1  0 0 300 30"

with open('positives.txt', 'r+') as the_file:
    the_file.write(pos_list)

In [0]:
# # создать файл со списком негативных картинок
neg_dir = "neg/"

neg_list = ""
for name in os.listdir(neg_dir):
  if name.endswith(".png"):
    if len(neg_list)<1:
      neg_list =  neg_dir + name
    else:
      neg_list = neg_list + "\r\n" +  neg_dir + name

with open('negatives.txt', 'r+') as the_file:
    the_file.write(neg_list)

In [0]:
# создать папку classifier для хаара, или очистить ее, а так же удалить вектор если есть
!rm /content/positives.vec
!rm -rf /content/classifier
!mkdir /content/classifier

In [0]:
# из списка positives создать вектор -w ширика картинки, -h высота картинки
#-w 300 -h 30
!opencv_createsamples -info "positives.txt" -vec "positives.vec" -num 500  -w 50 -h 30 

In [0]:
# обучить каскад -vec путь до файла позитивных векторов, -bg путь до списка негативных картиок, -w ширика картинки, -h высота картинки
# результаты сохранятся в папке classifier, если параметры изменяются - ее надо очистить
#-w 300 -h 30
!opencv_traincascade -data classifier -vec /content/positives.vec -bg /content/negatives.txt\
   -numStages 20 -minHitRate 0.999 -maxFalseAlarmRate 0.5 -numPos 500\
   -numNeg 500 -w 50 -h 30 -mode ALL -precalcValBufSize 1024\
   -precalcIdxBufSize 1024

In [0]:
# если обучение прервано раньше - чтоб скомпилировать готовый к использованию каскад
# нужно заменить numStages на то число эпох которые уже обучились и лежат в каталоге classifier

!opencv_traincascade -data classifier -vec /content/positives.vec -bg /content/negatives.txt\
   -numStages 3 -minHitRate 0.999 -maxFalseAlarmRate 0.5 -numPos 500\
   -numNeg 500 -w 50 -h 30 -mode ALL -precalcValBufSize 1024\
   -precalcIdxBufSize 1024

In [0]:
import os
from PIL import Image
import cv2
from IPython.display import display

def colab_imshow(frame):
    if os.path.exists('temp.jpg'):
        os.remove('temp.jpg')

    height, width = frame.shape[:2]
    frame = cv2.resize(frame, (500, int((500 / width) * height)), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('pic.jpg', frame)
    display(Image.open('pic.jpg'))
    os.remove('pic.jpg')

haarCascade = cv2.CascadeClassifier('/content/classifier/cascade.xml')

image = cv2.imread('/content/classifier/img11.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

haar_feed = haarCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(1, 1),
        maxSize=(50, 5)
) 

for (x, y, w, h) in haar_feed:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
colab_imshow(image)